In [1]:
from pathlib import Path
from typing import Union

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from utils.predictions import predict_img, convert_idx_img_to_color, get_dataset_generators

EVALUATION_DIR = Path("../results_json/cityscapes/")
EVALUATION_DIR.mkdir(parents=True, exist_ok=True)
BATCH_SIZE = 2
val_images_num = 500

2024-01-22 21:06:22.219312: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
models_dir = Path("../models/cityscapes/")

def _get_activation_from_model_name(model_name: Union[Path, str]) -> str:
    model_name_str = str(model_name) if isinstance(model_name, Path) else model_name

    return "_".join(model_name_str.split("_")[:-1])

In [3]:
activations, models = zip(*[(_get_activation_from_model_name(str(f).split("/")[-1]), str(f)) for f in models_dir.iterdir()])

In [4]:
_, val_set = get_dataset_generators(batch_size=BATCH_SIZE)

Found 2972 images belonging to 1 classes.
Found 2972 images belonging to 1 classes.
Found 500 images belonging to 1 classes.
Found 500 images belonging to 1 classes.


2024-01-22 21:06:26.872212: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-22 21:06:26.875832: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-22 21:06:26.876050: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-22 21:06:26.876738: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-22 21:06:26.876936: I tensorflow/compile

In [5]:
import dataclasses as dc
from json import dump

from keras.models import load_model
from tensorflow.keras.metrics import Recall, Precision

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

@dc.dataclass
class EvaluationResult:
    loss: float
    accuracy: float
    recall: float
    precision: float
    iou: float

def evaluate_model(model, val_set, name):
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy', Recall(name="recall"), Precision(name="precision"), iou])
    eval = model.evaluate(val_set, steps=val_images_num // BATCH_SIZE, return_dict=True)
    eval_res = EvaluationResult(**eval)
    with open(EVALUATION_DIR / f"{name}.json", "w") as f:
        dump(dc.asdict(eval_res), f)

In [6]:
activations, models = zip(*[(act, mod) for act, mod in zip(activations, models) if "modulus" not in mod])

In [7]:
for model_name, model in zip(activations, models):
    print(model_name)
    loaded_model = load_model(model)
    evaluate_model(loaded_model, val_set, model_name)

gelu


2024-01-22 21:06:46.905208: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600


250/250 [==============================] - 13s 36ms/step - loss: 0.5413 - accuracy: 0.8430 - recall: 0.8017 - precision: 0.9076 - iou: 0.6464
leaky_relu
250/250 [==============================] - 8s 29ms/step - loss: 0.5545 - accuracy: 0.8399 - recall: 0.7974 - precision: 0.9059 - iou: 0.6370
mish
250/250 [==============================] - 9s 33ms/step - loss: 0.5550 - accuracy: 0.8393 - recall: 0.7965 - precision: 0.9068 - iou: 0.6391
swish
250/250 [==============================] - 9s 32ms/step - loss: 0.5692 - accuracy: 0.8360 - recall: 0.7915 - precision: 0.9058 - iou: 0.6301
